# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../WeatherPy/output/cities.csv"

cities_df =  pd.read_csv(file)
cities_df

,cities,cloudiness,country,date,humidity,lat,lon,temp_max,temp_min,wind_speed
0,ushuaia,40,AR,1577941103,45,-54.81,-68.31,62.60,62.60,8.05
1,jardim,39,BR,1577941103,92,-21.48,-56.15,71.74,71.74,0.38
2,hermanus,69,ZA,1577940566,86,-34.42,19.24,64.99,62.01,1.01
3,mount isa,40,GB,1577941103,81,53.66,-1.85,44.01,39.99,12.75
4,monmouth,90,GB,1577941104,87,51.81,-2.72,46.99,43.00,5.82
...,...,...,...,...,...,...,...,...,...,...
243,satipo,100,PE,1577941123,96,-11.26,-74.64,60.40,60.40,2.28
244,shitanjing,0,CN,1577940592,48,39.23,106.34,30.88,30.88,3.85
245,zhob,86,PK,1577941124,44,31.34,69.45,42.40,42.40,4.79
246,ambam,95,CM,1577941124,90,2.38,11.27,71.38,71.38,3.13


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities_df[["lat", "lon"]]
humidity = cities_df["humidity"].astype(float)


In [4]:
fig = gmaps.figure(map_type="TERRAIN")

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [3]:
perfect_temp_min = cities_df["temp_min"]>72
perfect_wind = cities_df["wind_speed"]<10
perfect_clouds = cities_df["cloudiness"]<10



cities_df.where(perfect_temp_min & perfect_wind & perfect_clouds, inplace=True)






In [4]:
perfect_destination= cities_df.dropna(thresh=1)

print(len(perfect_destination))
perfect_destination.head(5)

4


,cities,cloudiness,country,date,humidity,lat,lon,temp_max,temp_min,wind_speed
14,carnarvon,0.0,ZA,1.577941e+09,35.0,-30.97,22.13,74.62,74.62,4.43
28,jamestown,0.0,AU,1.577941e+09,8.0,-33.21,138.60,93.16,93.16,9.01
185,champerico,5.0,MX,1.577941e+09,69.0,16.38,-93.60,75.20,75.20,4.70
234,nicoya,2.0,CR,1.577941e+09,69.0,10.15,-85.45,78.80,73.00,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = perfect_destination

#hotel_df.set_index("cities")

In [3]:
cities_df["Hotel Name"] = ""
#hotel_df.set_index(["cities"])
cities_df.head()

,cities,cloudiness,country,date,humidity,lat,lon,temp_max,temp_min,wind_speed,Hotel Name
0,ushuaia,40,AR,1577941103,45,-54.81,-68.31,62.60,62.60,8.05,
1,jardim,39,BR,1577941103,92,-21.48,-56.15,71.74,71.74,0.38,
2,hermanus,69,ZA,1577940566,86,-34.42,19.24,64.99,62.01,1.01,
3,mount isa,40,GB,1577941103,81,53.66,-1.85,44.01,39.99,12.75,
4,monmouth,90,GB,1577941104,87,51.81,-2.72,46.99,43.00,5.82,


In [5]:


#perfect_loc = perfect_destination[["lat", "lon"]]
params = {
    #"location": "hotel_df['lat'],hotel_df['lon']",  
    "type": "lodging",
    "key": g_key,
    "radius": 5000
}


In [10]:
for index, row in cities_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    plat = row["lat"]
    plon = row["lon"]
    params["location"]=f"{plat},{plon}"
    
    hotel_response=requests.get(base_url,params=params).json()
    results = hotel_response['results']
    try:
        cities_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("no luck")

print(results[0]['name'])
print(hotel_df)  

no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck
no luck


KeyboardInterrupt: 

In [11]:

print(hotel_df) 

         cities  cloudiness country        date  humidity    lat     lon  \
14    carnarvon           0      ZA  1577940556        35 -30.97   22.13   
28    jamestown           0      AU  1577941105         8 -33.21  138.60   
185  champerico           5      MX  1577941119        69  16.38  -93.60   
234      nicoya           2      CR  1577941123        69  10.15  -85.45   

     temp_max  temp_min  wind_speed Hotel Name  
14      74.62     74.62        4.43             
28      93.16     93.16        9.01             
185     75.20     75.20        4.70             
234     78.80     73.00        9.17             


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map